Testing the deep forest module on a part of the SVMA data

Instructions: https://deepforest.readthedocs.io/en/latest/getting_started.html#issues

Modifications:
- first run throws an error regarding incompatible torch/torchvision. Reinstalled after installing deep forest using the following command in anaconda prompt, correct environmnt:
uninstall torch torchvision && pip install torch torchvision --no-cache-dir

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

import rioxarray

In [ ]:
data_image = Path('../../data/Fieldwork_data/PNM/gpkg_borore/area_recortada_mosaico.tif')
image_uav = rioxarray.open_rasterio(data_image).isel(band=range(3))

In [ ]:
image_uav

### Split a big image to crops

In [ ]:
from deepforest_exp import make_crops

# Define crops by tif coordinates
crop_centers = [(331_050,7366_550), (331_050,7366_660), (331_150,7366_650), (331_250,7366_800), (331_350,7366_810), (331_400,7366_900)]
crop_sizes = 55

img_crops = make_crops(image_uav, crop_centers, crop_sizes)

### Predict tree crowns on crops

In [ ]:
# Predict all crops with one set of parameters
from deepforest_exp import predict_crops

tile_size = 500
overlap_ratio = 0.45

gdf_prediction = predict_crops(img_crops, tile_size, overlap_ratio)

In [ ]:
gdf_prediction

In [ ]:
fig, axes = plt.subplots(2,3)
fig.set_size_inches(15, 9)
for img, ax in zip(img_crops, axes.flatten()):
    xmin, ymin, xmax, ymax = img.rio.bounds()
    predicts = gdf_prediction.cx[xmin:xmax, ymin:ymax]
    img.plot.imshow(ax=ax)
    predicts.plot(ax=ax, facecolor="none", edgecolor="red")
    ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

### Predict on crops, trest combinations of different parameters

In [ ]:
# Predict all crops with one set of parameters
from deepforest_exp import predict_experiments

# Define param search space
tile_sizes = [500, 600] 
overlap_ratios = [0.35, 0.45]

# Predict for all combinations
predicts_experiments = predict_experiments(img_crops, tile_sizes, overlap_ratios)

In [ ]:
predicts_experiments